In [ ]:
# Cell 1: Imports
import sys
import os
import joblib
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import json

# Cell 2: Parameters
# tags=["parameters"]
INPUT_SEMI_PATH = '../data/processed/dataset_semi_supervised.parquet'
METRICS_OUT = '../data/processed/co_training_metrics.json'
CONFIDENCE_THRESHOLD = 0.8  # Chỉ tin tưởng gán nhãn nếu xác suất > 80%
MAX_ITERATIONS = 5           # Số vòng lặp Co-training

# Cell 3: Main Execution
print("--- BẮT ĐẦU: CO-TRAINING (MULTI-VIEW LEARNING) ---")

# 1. Load Data
if not os.path.exists(INPUT_SEMI_PATH):
    raise FileNotFoundError("Chưa chạy bước semi_dataset_preparation!")

data = joblib.load(INPUT_SEMI_PATH)
X_train = data['X_train']
y_train_semi = data['y_train_semi'].copy() # Copy để không sửa vào dữ liệu gốc
X_test = data['X_test']
y_test = data['y_test']

print(f"Tổng mẫu Train: {len(X_train)}")
print(f"Số mẫu ban đầu chưa nhãn (-1): {sum(y_train_semi == -1)}")

# 2. Định nghĩa 2 Views (Góc nhìn)
# View 1: Các chất ô nhiễm (Chemicals)
view1_cols = ['PM10', 'SO2', 'NO2', 'CO', 'O3']
# View 2: Khí tượng (Meteorology)
view2_cols = ['TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM']

# Kiểm tra xem các cột có đủ không
available_cols = X_train.columns.tolist()
v1_real = [c for c in view1_cols if c in available_cols]
v2_real = [c for c in view2_cols if c in available_cols]

print(f"View 1 features: {v1_real}")
print(f"View 2 features: {v2_real}")

# 3. Khởi tạo 2 classifiers riêng biệt
clf1 = RandomForestClassifier(n_estimators=50, random_state=1, n_jobs=-1)
clf2 = RandomForestClassifier(n_estimators=50, random_state=2, n_jobs=-1)

# 4. Vòng lặp Co-Training thủ công
# Logic: Clf1 học trên view1 -> gán nhãn dữ liệu khó -> đưa cho Clf2 học và ngược lại

for i in range(MAX_ITERATIONS):
    print(f"\n--- Iteration {i+1}/{MAX_ITERATIONS} ---")
    
    # Lấy các mẫu ĐÃ CÓ NHÃN (khác -1) để train
    mask_labeled = y_train_semi != -1
    X_labeled = X_train[mask_labeled]
    y_labeled = y_train_semi[mask_labeled]
    
    # Lấy các mẫu CHƯA CÓ NHÃN (-1) để dự đoán
    mask_unlabeled = y_train_semi == -1
    if sum(mask_unlabeled) == 0:
        print("Đã gán nhãn hết toàn bộ dữ liệu!")
        break
        
    X_unlabeled = X_train[mask_unlabeled]
    original_indices = X_unlabeled.index # Lưu index gốc để gán lại
    
    # Train 2 model trên 2 view
    clf1.fit(X_labeled[v1_real], y_labeled)
    clf2.fit(X_labeled[v2_real], y_labeled)
    
    # Dự đoán xác suất trên tập chưa nhãn
    prob1 = clf1.predict_proba(X_unlabeled[v1_real])
    prob2 = clf2.predict_proba(X_unlabeled[v2_real])
    
    # Lấy nhãn dự đoán và độ tin cậy cao nhất
    pred1 = prob1.argmax(axis=1)
    conf1 = prob1.max(axis=1)
    
    pred2 = prob2.argmax(axis=1)
    conf2 = prob2.max(axis=1)
    
    # Tìm những mẫu mà Model 1 rất tự tin -> Gán nhãn -> Thêm vào tập train
    newly_labeled_count = 0
    
    # Logic gán nhãn: Nếu Conf > Threshold thì cập nhật y_train_semi
    # Ở đây dùng logic đơn giản: Duyệt qua các mẫu unlabel
    
    updates = []
    
    # Vector hóa việc update để chạy nhanh hơn
    # Cập nhật từ View 1
    high_conf_idx1 = np.where(conf1 >= CONFIDENCE_THRESHOLD)[0]
    for idx in high_conf_idx1:
        real_idx = original_indices[idx]
        if y_train_semi[real_idx] == -1: # Chỉ gán nếu chưa ai gán trong vòng lặp này
            y_train_semi[real_idx] = pred1[idx]
            updates.append(real_idx)
            
    # Cập nhật từ View 2
    high_conf_idx2 = np.where(conf2 >= CONFIDENCE_THRESHOLD)[0]
    for idx in high_conf_idx2:
        real_idx = original_indices[idx]
        if y_train_semi[real_idx] == -1: 
            y_train_semi[real_idx] = pred2[idx]
            updates.append(real_idx)
            
    new_count = len(updates)
    print(f"Số mẫu được gán nhãn mới (Pseudo-labeled): {new_count}")
    
    if new_count == 0:
        print("Không tìm thấy mẫu nào đủ độ tin cậy để gán thêm. Dừng sớm.")
        break

# 5. Huấn luyện lần cuối và Đánh giá (Ensemble 2 views)
print("\n--- FINAL EVALUATION ---")
# Train lại trên toàn bộ dữ liệu (gốc + giả nhãn)
clf1.fit(X_train[y_train_semi != -1][v1_real], y_train_semi[y_train_semi != -1])
clf2.fit(X_train[y_train_semi != -1][v2_real], y_train_semi[y_train_semi != -1])

# Dự đoán trên Test set (lấy trung bình cộng xác suất 2 model)
p1_test = clf1.predict_proba(X_test[v1_real])
p2_test = clf2.predict_proba(X_test[v2_real])
p_avg = (p1_test + p2_test) / 2
y_final_pred = p_avg.argmax(axis=1)

acc = accuracy_score(y_test, y_final_pred)
print(f"Co-Training Final Accuracy: {acc:.4f}")
print(classification_report(y_test, y_final_pred))

# 6. Lưu kết quả
results = {
    "co_training_accuracy": acc,
    "iterations_run": i + 1,
    "final_labeled_count": int(sum(y_train_semi != -1))
}

with open(METRICS_OUT, 'w') as f:
    json.dump(results, f)
print(f"Đã lưu metrics tại: {METRICS_OUT}")